In [1]:
#Importation des librairies
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import json

In [17]:
#L'URL de base nous permet d'avoir l'ensemble des résultats
base_url = "https://www.rottentomatoes.com/browse/movies_at_home/sort:popular?page=40"

#initialisation de la liste où on va stocker tous les liens des pages en particulier
links=[]

#fonction qui scrape l'ensemble des liens sur la page
def scraper_links(base_url):
    rs = requests.Session() #ouverture d'une session request
    response = rs.get(base_url) 
    soup = BeautifulSoup(response.text,'html5lib') #on prend le texte du request de toute la page et on en tire le html pour le stocker dans une soupe
    for movie in soup.find_all("a",{"class":"js-tile-link"}): #on target la classe HTML où se trouve l'URL de chaque film
        link = movie.get('href') #chaque URL est dans href 
        links.append(link)
    return links
    

movie_links = scraper_links(base_url) #on affecte à une variable la fonction qui nous permet de retourner une partie de l'URL des films

In [18]:
len(movie_links) 

151

In [22]:
href_links = [i for i in movie_links if i] #removing none values
whole_links = [("https://www.rottentomatoes.com"+i) for  i in href_links] #on construit une liste avec tous les liens entiers, y compris la racine
whole_links

['https://www.rottentomatoes.com/m/puss_in_boots_the_last_wish',
 'https://www.rottentomatoes.com/m/the_menu',
 'https://www.rottentomatoes.com/m/the_banshees_of_inisherin',
 'https://www.rottentomatoes.com/m/the_pale_blue_eye',
 'https://www.rottentomatoes.com/m/everything_everywhere_all_at_once',
 'https://www.rottentomatoes.com/m/the_fabelmans',
 'https://www.rottentomatoes.com/m/the_hatchet_wielding_hitchhiker',
 'https://www.rottentomatoes.com/m/glass_onion_a_knives_out_mystery',
 'https://www.rottentomatoes.com/m/sick_2022',
 'https://www.rottentomatoes.com/m/mars_one',
 'https://www.rottentomatoes.com/m/theres_something_wrong_with_the_children',
 'https://www.rottentomatoes.com/m/devotion_2022',
 'https://www.rottentomatoes.com/m/aftersun',
 'https://www.rottentomatoes.com/m/white_noise_2022',
 'https://www.rottentomatoes.com/m/emily_the_criminal',
 'https://www.rottentomatoes.com/m/the_offering_2022',
 'https://www.rottentomatoes.com/m/dont_worry_darling',
 'https://www.rottent

In [24]:
#définit une fonction de fermeture des cookies propre au site rotten tomatoes - on refuse les cookies ! 
def cookie_closer(first_page):
    time.sleep(4) #on laisse la page en veille pendant 4 secondes : cela permet au pop-up cookies de s'afficher
    if first_page:
        close_cookies = driver.find_element(By.ID, "onetrust-reject-all-handler") #on cherche l'élément par l'ID
        close_cookies.click() #on clique sur le bouton fermeture des cookies
        time.sleep(3)

In [37]:
#On définit une fonction qu'on appellera plus tard, qui nous permet d'avoir toutes les informations que l'on souhaite avoir
def info_getter():
    titles.append(driver.find_element(By.XPATH,'//*[@id="topSection"]/div[1]/score-board/h1').text) #permet d'avoir le titre des films
    infos.append(driver.find_element(By.XPATH,'//*[@id="topSection"]/div[1]/score-board/p').text) #permet d'avoir les informations rapides : durée, langue et genre du film
    score_board = driver.find_element(By.XPATH,'//*[@id="topSection"]/div[1]/score-board') #permet d'avoir le scoreboard : on trouvera plusieurs informations, tels que les scores ou encore des "states"
    #étant donné qu'on n'a pas toujours toutes les informations concernant le score, surtout pour les films récents, on teste le bloc de code. 
    #si on ne trouve pas l'information, on retourne une valeur vide
    try:
        audiencestates.append(score_board.get_attribute('audiencestate')) #l'avis résumé en une expression de l'audience
    except:
        audiencestates.append('empty')
    try:
        tomatometersstates.append(score_board.get_attribute('tomatometerstate')) #l'avis résumé en une expression du site
    except:
        tomatometersstates.append('empty')
    try:
        tomatometers.append(score_board.get_attribute('tomatometerscore')) #le score du film donné par rotten tomatoes
    except:
        tomatometers.append('empty')
    try:
        audiencescores.append(score_board.get_attribute('audiencescore')) #le score du film donné par l'audience
    except: 
        audiencescores.append('empty')
    movie_infos.append(driver.find_element(By.CSS_SELECTOR,'#mainColumn > section.panel.panel-rt.panel-box.movie_info.media > div > div').text) #l'ensemble des informations détaillées du film 
    try:
        critics_consensus.append(driver.find_element(By.XPATH, '//*[@id="what-to-know"]/div/section/p/span').text) #l'avis général du film
    except: 
        critics_consensus.append('empty')
    try:
        critics_hrefs.append(driver.find_element(By.XPATH, '//*[@id="criticReviewsModule"]/div/div/div[2]/a').get_attribute('href'))
    except:
        critics_hrefs.append('empty')

In [38]:
driver = webdriver.Chrome() #on ouvre le webdriver chrome 

big_soup, titles, infos, audiencestates, audiencescores, tomatometersstates, tomatometers, movie_infos, critics_consensus, critics_hrefs = ([] for i in range(10)) #affectation de 10 listes vides
elements = {} #affectation d'un dictionnaire vide où on va stocker nos listes
first_page = True #on part de l'hypothèse qu'on est d'abord sur une première page

for link in whole_links: #pour chaque lien dans la liste de liens 
    driver.get(link) #le driver chrome va aprendre chaque lien

    cookie_closer(first_page) #on va fermer les cookies
    first_page = False #maintenant qu'on a fermé les cookies, le driver ne change pas de session et retient nos préférences 
    
    page = driver.page_source #on prend le page source des drivers
    detail_soup = BeautifulSoup(page) #on prend toutes les informations 
    big_soup.append(detail_soup)

    info_getter() #appel de la fonction qui nous permet d'avoir toutes les informations

    elements = {'titles':titles,
                'infos':infos,
                'audience state':audiencestates,
                'audience scores':audiencescores,
                'tomatometer state':tomatometersstates,
                'tomatometer':tomatometers,
                'movie infos':movie_infos, 
                'critics consensus':critics_consensus,
                'critics links': critics_hrefs}

driver.quit() #on ferme le driver 
elements #on a le dictionnaire final qui va nous permettre de créer la dataframe

{'titles': ['PUSS IN BOOTS: THE LAST WISH',
  'THE MENU',
  'THE BANSHEES OF INISHERIN',
  'THE PALE BLUE EYE',
  'EVERYTHING EVERYWHERE ALL AT ONCE',
  'THE FABELMANS',
  'THE HATCHET WIELDING HITCHHIKER',
  'GLASS ONION: A KNIVES OUT MYSTERY',
  'SICK',
  'MARS ONE',
  "THERE'S SOMETHING WRONG WITH THE CHILDREN",
  'DEVOTION',
  'AFTERSUN',
  'WHITE NOISE',
  'EMILY THE CRIMINAL',
  'THE OFFERING',
  "DON'T WORRY DARLING",
  'DOG GONE',
  'DETECTIVE KNIGHT: INDEPENDENCE',
  'BULLET TRAIN',
  'BLACK ADAM',
  'AMSTERDAM',
  'THE OLD WAY',
  'X',
  'BARBARIAN',
  'BLAZE',
  'WHERE THE CRAWDADS SING',
  'ALL QUIET ON THE WESTERN FRONT',
  'TOP GUN: MAVERICK',
  'AVATAR',
  'ELVIS',
  "GUILLERMO DEL TORO'S PINOCCHIO",
  'VENGEANCE',
  'NOISE',
  'THE WOMAN KING',
  'THE WONDER',
  'STRANGE WORLD',
  'NOPE',
  'PUSS IN BOOTS',
  'HELLBENDER',
  'TO LESLIE',
  'GUNFIGHT AT RIO BRAVO',
  'THE INNOCENTS',
  'SMILE',
  'KNIVES OUT',
  'JURASSIC WORLD DOMINION',
  'THE NORTHMAN',
  'ON THE LINE

In [62]:
best_movies_at_home = pd.DataFrame(elements) #dataframe finale à partir du dictionnaire
best_movies_at_home

,titles,infos,audience state,audience scores,tomatometer state,tomatometer,movie infos,critics consensus,critics links
0,PUSS IN BOOTS: THE LAST WISH,"2022, Kids & family/Comedy, 1h 42m",upright,93,certified-fresh,95,"This fall, everyone's favorite leche-loving, s...",Arriving more than a decade after the previous...,https://www.rottentomatoes.com/m/puss_in_boots...
1,THE MENU,"2022, Horror/Mystery & thriller, 1h 47m",upright,76,certified-fresh,88,A couple (Anya Taylor-Joy and Nicholas Hoult) ...,While its social commentary relies on basic in...,https://www.rottentomatoes.com/m/the_menu/reviews
2,THE BANSHEES OF INISHERIN,"2022, Comedy, 1h 49m",upright,76,certified-fresh,97,Set on a remote island off the west coast of I...,Featuring some of Martin McDonagh's finest wor...,https://www.rottentomatoes.com/m/the_banshees_...
3,THE PALE BLUE EYE,"2022, Mystery & thriller/Drama, 2h 10m",upright,72,fresh,62,"West Point, 1830. In the early hours of a gray...",The Pale Blue Eye lacks its source material's ...,https://www.rottentomatoes.com/m/the_pale_blue...
4,EVERYTHING EVERYWHERE ALL AT ONCE,"2022, Comedy/Adventure, 2h 12m",upright,89,certified-fresh,95,"Directed by Daniel Kwan and Daniel Scheinert, ...","Led by an outstanding Michelle Yeoh, Everythin...",https://www.rottentomatoes.com/m/everything_ev...
...,...,...,...,...,...,...,...,...,...
145,AMERICAN PSYCHO,"2000, Comedy/Mystery & thriller, 1h 43m",upright,85,fresh,68,"In New York City in 1987, a handsome, young ur...",If it falls short of the deadly satire of Bret...,https://www.rottentomatoes.com/m/american_psyc...
146,PARASITE,"2019, Comedy/Mystery & thriller, 2h 12m",upright,90,certified-fresh,99,Greed and class discrimination threaten the ne...,"An urgent, brilliantly layered look at timely ...",https://www.rottentomatoes.com/m/parasite_2019...
147,PADDINGTON 2,"2017, Kids & family/Comedy, 1h 43m",upright,88,certified-fresh,99,"Settled in with the Brown family, Paddington t...",Paddington 2 honors its star's rich legacy wit...,https://www.rottentomatoes.com/m/paddington_2/...
148,CAUSEWAY,"2022, Drama, 1h 32m",upright,74,certified-fresh,86,"""Causeway"" is an intimate portrait of a soldie...",Causeway takes a powerfully subdued look at th...,https://www.rottentomatoes.com/m/causeway/reviews


In [44]:
best_movies_at_home['audience state'].value_counts()

upright    124
spilled     23
             3
Name: audience state, dtype: int64

Nous savons que la colonne movie infos contient des caractères montrant un retour à la ligne. Etant donné que nous allons procéder au feature engineering des données sur pyspark et que pyspark reconnait uniquement les \n comme des retours à la ligne dans le dataframe spark, on remplace chaque \n par des \t. 

In [63]:
best_movies_at_home = best_movies_at_home.replace(r'\n','\t', regex=True) 
best_movies_at_home['movie infos'][0]

'This fall, everyone\'s favorite leche-loving, swashbuckling, fear-defying feline returns. For the first time in more than a decade, DreamWorks Animation presents a new adventure in the Shrek universe as daring outlaw Puss in Boots discovers that his passion for peril and disregard for safety have taken their toll. Puss has burned through eight of his nine lives, though he lost count along the way. Getting those lives back will send Puss in Boots on his grandest quest yet. Academy Award® nominee Antonio Banderas returns as the voice of the notorious PiB as he embarks on an epic journey into the Black Forest to find the mythical Wishing Star and restore his lost lives. But with only one life left, Puss will have to humble himself and ask for help from his former partner and nemesis: the captivating Kitty Soft Paws (Oscar® nominee Salma Hayek). In their quest, Puss and Kitty will be aided--against their better judgment--by a ratty, chatty, relentlessly cheerful mutt, Perro (Harvey Guillé

In [64]:
best_movies_at_home.to_csv('best_movies_at_home.csv',sep=';',index=False) #export en CSV qui enlève une colonne propre à l'index. 